<a href="https://colab.research.google.com/github/super-onigiri-man/JOEU-FM/blob/main/FM_%E3%83%99%E3%82%B9%E3%83%88%E3%83%92%E3%83%83%E3%83%88%E3%83%A9%E3%83%B3%E3%82%AD%E3%83%B3%E3%82%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

詳しい使用方法は[こちら](https://github.com/super-onigiri-man/JOEU-FM/wiki)

よく出るエラーの詳細は[こちら](https://github.com/super-onigiri-man/JOEU-FM/wiki/%E3%82%A8%E3%83%A9%E3%83%BC%E3%81%8B%E3%81%AA%EF%BC%9F%E3%81%A8%E6%80%9D%E3%81%A3%E3%81%9F%E3%82%89)

上から一つずつ実行していってください。

In [ ]:
!pip install openpyxl #Excelデータの読み書き
!pip install mojimoji #全角文字を半角文字にするためのモジュール

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.4/192.4 kB 4.2 MB/s eta 0:00:00


In [ ]:
import sqlite3 #データベースを動かすためのモジュール
import pandas as pd #ランキングを作成するためのモジュール
import openpyxl #Excelから情報を取ってくるモジュール
import mojimoji #全角文字を半角文字に変換するためのモジュール



**楽曲データ.csvからデータベースを作成**

In [ ]:
import csv
import sqlite3

dbname = ('test.db')#データベース名.db拡張子で設定
conn = sqlite3.connect(dbname, isolation_level=None)#データベースを作成、自動コミット機能ON
cursor = conn.cursor() #カーソルオブジェクトを作成
sql = """CREATE TABLE music_master (
    Title TEXT,
    Artist TEXT,
    Score DOUBLE,
    Last_Rank INT,
    Last_Number INT,
    On_Chart INT,
    PRIMARY KEY (Title, Artist)
);"""
#↑DBのフォーマット設定（別のところに書いておきます）
cursor.execute(sql)#executeコマンドでSQL文を実行
conn.commit()#データベースにコミット(Excelでいう上書き保存。自動コミット設定なので不要だが一応・・)
with open('楽曲データ.csv', 'r') as f:
    # CSVリーダーオブジェクトを作成
    csv_reader = csv.reader(f)
    for row in csv_reader:
        # テーブルにデータを挿入
        cursor.execute("INSERT INTO music_master VALUES (?,?,?,?,?,?)", (row[0], row[1],row[2],row[3],row[4],row[5]))

**各サイトからランキングを取得**

In [ ]:
import requests
from bs4 import BeautifulSoup #スクレイピング（取得）用
import datetime #日付計算用

#scoreを元にしたオリコンシングル・オリコンデジタル・ビルボード総合ランキング

# オリコン週間ランキング用
OriconWeekData = []
# オリコンデジタルランキング用
OriconDigitalData = []
# ビルボードランキング用
BillboardData = []
# 明屋書店ランキング用
HaruyaData = []


def selecter():

    print("見たい週を選択してください")
    select = int(input("1:今回 2:前回"))
    return select

def OriconTodays():
    # 今日の日付を求める
    d_today = datetime.date.today()
    # 今日の曜日を求める
    todayweek = datetime.date.today().weekday()
    #print(d_today, todayweek)

    #オリコンの発表は毎週水曜日のため、火曜日までは先週のランキングを表示
    #日付は来週の水曜日付となる。
    if (todayweek == 0):  # 今日が月曜日(先週(今週月曜日)のランキング表示)
        Oriconday = d_today
    elif (todayweek == 1):# 今日が火曜日(先週(今週月曜日)のランキング表示)
        Oriconday = d_today - datetime.timedelta(days=1)
    elif (todayweek == 2):# 今日が水曜日
        Oriconday = d_today + datetime.timedelta(days=5)
    elif (todayweek == 3):# 今日が木曜日
        Oriconday = d_today + datetime.timedelta(days=4)
    elif (todayweek == 4):# 今日が金曜日
        Oriconday = d_today + datetime.timedelta(days=3)
    elif (todayweek == 5):# 今日が土曜日
        Oriconday = d_today + datetime.timedelta(days=2)
    elif (todayweek == 6):# 今日が日曜日
        Oriconday = d_today + datetime.timedelta(days=1)

    return Oriconday


def OriconLastWeek():
    # 今日の日付を求める
    d_today = datetime.date.today()
    # 今日の曜日を求める
    todayweek = datetime.date.today().weekday()
    # print(d_today, todayweek)

    # オリコンの発表は毎週水曜日のため、火曜日までは先週のランキングを表示
    # 日付は来週の月曜日付となる。
    if (todayweek == 0):  # 今日が月曜日(先週(今週月曜日)のランキング表示)
        Oriconday = d_today - datetime.timedelta(days=7)
    elif (todayweek == 1):  # 今日が火曜日(先週(今週月曜日)のランキング表示)
        Oriconday = d_today - datetime.timedelta(days=8)
    elif (todayweek == 2):  # 今日が水曜日
        Oriconday = d_today - datetime.timedelta(days=2)
    elif (todayweek == 3):  # 今日が木曜日
        Oriconday = d_today - datetime.timedelta(days=3)
    elif (todayweek == 4):  # 今日が金曜日
        Oriconday = d_today - datetime.timedelta(days=4)
    elif (todayweek == 5):  # 今日が土曜日
        Oriconday = d_today - datetime.timedelta(days=5)
    elif (todayweek == 6):  # 今日が日曜日
        Oriconday = d_today - datetime.timedelta(days=6)

    return Oriconday


def OriconSelectWeek():
    # 日付の入力を促す
    date = input("2020年8月3日以降の日付を入力してください (YYYY-MM-DD): ")

    # 入力された日付をdatetimeオブジェクトに変換
    dt = datetime.datetime.strptime(date, "%Y-%m-%d")
    dt = dt.date()
    # 曜日を取得
    weekday = dt.weekday()

    # オリコンの発表は毎週水曜日のため、火曜日までは先週のランキングを表示
    # 日付は来週の月曜日付となる。
    if (weekday == 0):  # 今日が月曜日
        Oriconday = dt
    elif (weekday == 1):  # 今日が火曜日
        Oriconday = dt - datetime.timedelta(days=1)
    elif (weekday == 2):  # 今日が水曜日
        Oriconday = dt - datetime.timedelta(days=2)
    elif (weekday == 3):  # 今日が木曜日
        Oriconday = dt - datetime.timedelta(days=3)
    elif (weekday == 4):  # 今日が金曜日
        Oriconday = dt - datetime.timedelta(days=4)
    elif (weekday == 5):  # 今日が土曜日
        Oriconday = dt - datetime.timedelta(days=5)
    elif (weekday == 6):  # 今日が日曜日
        Oriconday = dt - datetime.timedelta(days=6)

    return Oriconday


def OriconWeekRank(Oriconday):#オリコン週間ランキング

    #1位から10位
    load_url = "https://www.oricon.co.jp/rank/js/w/" + str(Oriconday) + "/"
    html = requests.get(load_url)
    soup = BeautifulSoup(html.text, "html.parser")
    links = soup.find(class_="content-rank-main").find_all('h2',class_='title') #曲名
    artist = soup.find(class_="content-rank-main").find_all('p',class_='name') #アーティスト名
    score = 6.0 #独自スコア
    rank = 1 #ランキング
    print(str(Oriconday) + "付けオリコン週間シングルランキング")

    index = 0
    for link, artist in zip(links, artist):
      if '/' in link.text:
        # If yes, split the title and create two separate entries in the array
        titles = link.text.split('/')
        for title in titles:
            mojimoji.zen_to_han(title.strip())  # Strip to remove leading/trailing whitespaces
            mojimoji.zen_to_han(artist.text)
            OriconWeekData.append([title.strip(), artist.text, "{:.1f}".format(score)])

            if title == titles[-1]:
              rank = rank + 1
              score = score - 0.3
      else:
        # If no slash, just add the entry to the array
        mojimoji.zen_to_han(link.text)
        mojimoji.zen_to_han(artist.text)
        OriconWeekData.append([link.text, artist.text, "{:.1f}".format(score)])
        rank = rank + 1
        score = score - 0.3

      # 壊れたときの表示用
      # if rank != 10:#10位以下（１ケタの場合）なら（点数の位置を揃えるため）
      #   print(" " + str(rank) + "位 " + "{:.1f}　 ".format(score) + link.text + "/" + artist.text)
      # else: #10位（２ケタの場合）なら
      #   print(str(rank) + "位 " + "{:.1f}　 ".format(score) + link.text + "/" + artist.text)

    print(OriconWeekData)

    #11位から20位
    load_url = "https://www.oricon.co.jp/rank/js/w/" + str(Oriconday) + "/p/2/"
    html = requests.get(load_url)
    soup = BeautifulSoup(html.text, "html.parser")
    links = soup.find(class_="content-rank-main").find_all('h2', class_='title')  # 曲名
    artist = soup.find(class_="content-rank-main").find_all('p', class_='name')  # アーティスト名
    for link, artist in zip(links, artist):
      if '/' in link.text:
        # 「楽曲A/楽曲B」のような表現方法の場合
        titles = link.text.split('/') #"/"で２曲に分ける
        for title in titles:
            mojimoji.zen_to_han(title.strip())  # Strip to remove leading/trailing whitespaces
            mojimoji.zen_to_han(artist.text)
            OriconWeekData.append([title.strip(), artist.text, "{:.1f}".format(score)])

            if title == titles[-1]:
              rank = rank + 1
              score = score - 0.3
      else:
        # If no slash, just add the entry to the array
        mojimoji.zen_to_han(link.text)
        mojimoji.zen_to_han(artist.text)
        OriconWeekData.append([link.text, artist.text, "{:.1f}".format(score)])
        rank = rank + 1
        score = score - 0.3
        # 壊れたときの表示用
      # print(str(rank) + "位 " + "{:.1f}　 ".format(score) + link.text + "/" + artist.text)


    # print(OriconWeekRank)



def OriconDigitalRank(Oriconday):#オリコンデジタルシングルランキング
    # 1位から10位
    load_url = "https://www.oricon.co.jp/rank/dis/w/" + str(Oriconday) + "/"
    html = requests.get(load_url)
    soup = BeautifulSoup(html.text, "html.parser")
    links = soup.find(class_="content-rank-main").find_all('h2', class_='title')
    artist = soup.find(class_="content-rank-main").find_all('p', class_='name')  # アーティスト名
    rank = 1
    score = float(6.0)
    print(str(Oriconday) + "付けオリコン週間デジタルシングルランキング")
    for link, artist in zip(links, artist):
      # 壊れたときの表示用
        # if rank != 10:#10位以下（１ケタの場合）なら（点数の位置を揃えるため）
        #     print(" " + str(rank) + "位 " + "{:.1f}　 ".format(score) + link.text + "/" + artist.text)
        # else: #10位（２ケタの場合）なら
        #     print(str(rank) + "位 " + "{:.1f}　 ".format(score) + link.text + "/" + artist.text)
        mojimoji.zen_to_han(link.text)
        mojimoji.zen_to_han(artist.text)
        OriconDigitalData.append([link.text,artist.text,"{:.1f}".format(score)])
        rank = rank + 1
        score = score - 0.3

    # 11位から20位
    load_url = "https://www.oricon.co.jp/rank/dis/w/" + str(Oriconday) + "/p/2/"
    html = requests.get(load_url)
    soup = BeautifulSoup(html.text, "html.parser")
    links = soup.find(class_="content-rank-main").find_all('h2', class_='title')
    artist = soup.find(class_="content-rank-main").find_all('p', class_='name')  # アーティスト名
    for link, artist in zip(links, artist):
        OriconDigitalData.append([link.text,artist.text,"{:.1f}".format(score)])
        # print(str(rank) + "位 " + "{:.1f}　 ".format(score) + link.text + "/" + artist.text)
        rank = rank + 1
        score = score - 0.3

def BillboadRank(Oriconday):#ビルボードJAPAN HOT100ランキング

    # オリコンの日付とビルボードの発表日の差を埋めるための計算
    Billday = Oriconday - datetime.timedelta(days=5)

    print(str(Billday) + "付けビルボードJAPAN HOT100ランキング")

    #URL(ここを変更すると読み込まなくなります)
    url = 'https://www.billboard-japan.com/charts/detail?a=hot100&year='+str(Oriconday.year)+'&month='+str(Oriconday.month)+'&day='+str(Oriconday.day)
    #URLを取得してくる
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    songs = soup.find_all('p', class_='musuc_title') #曲名
    artists = soup.find_all('p', class_='artist_name') #アーティスト名
    score = 6.0 #基準点

    for i in range(20):#20回繰り替えす
      song = songs[i].text.strip()
      artist = artists[i].text.strip()
      mojimoji.zen_to_han(song)
      mojimoji.zen_to_han(artist)
      BillboardData.append([song,artist,format(score, '.1f')])
      # if i < 9:
      #   print(f" {i+1}位: {format(score, '.1f')} {song} / {artist}") #1位から9位までのランキング
      # else:
      #   print(f"{i+1}位: {format(score, '.1f')} {song} / {artist}") #10位から20位までのランキング
      score = score - 0.3 #scoreを-0.3する


sel = selecter()


if sel==1: #今週のランキング表示
    OriconTodays()
    OriconWeekRank(OriconTodays())
    print()
    OriconDigitalRank(OriconTodays())
    print()
    BillboadRank(OriconTodays())

elif sel == 2: #前週のランキング表示
    OriconLastWeek()
    OriconWeekRank(OriconLastWeek())
    # print(OriconWeekData)
    print()
    OriconDigitalRank(OriconLastWeek())
    # print(OriconDigitalData)
    print()
    BillboadRank(OriconLastWeek())
    # print(BillboardData)

else : #別の週のランキング
    OriconWeekRank(OriconSelectWeek())
    print()
    OriconDigitalRank(OriconSelectWeek())
    print()
    BillboadRank(OriconSelectWeek())


見たい週を選択してください
1:今回 2:前回 3:別の回１
2024-02-26付けオリコン週間シングルランキング
[['LOVE TRIGGER', 'Snow Man', '6.0'], ['We’ll go together', 'Snow Man', '6.0'], ['Frozen Butterfly', 'WOLF HOWL HARMONY from EXILE TRIBE', '5.7'], ['Special Thanks', '', '5.4'], ['青とシャボン', '', '5.4'], ['ミルク', '', '5.4'], ['戦いが消えた日', 'SAD originals.', '5.1'], ['CRAZY CIRCUIT', 'D.O.L', '4.8'], ['君は何を後悔するのか?', 'STU48', '4.5'], ['After you!', 'キョコロヒー', '4.2'], ['HEARTBREAKER', 'Kis-My-Ft2', '3.9'], ['C‘monova', 'Kis-My-Ft2', '3.9'], ['アンビバレント', 'Uru', '3.6'], ['バケツを被れ!', 'HKT48', '3.3']]

2024-02-26付けオリコン週間デジタルシングルランキング

2024-02-21付けビルボードJAPAN HOT100ランキング


**明屋書店のExcelデータから情報を取得**

In [ ]:

# Excelファイルを読み込む
wb = openpyxl.load_workbook("0219ベストヒット愛媛.xlsx")
ws = wb.active

# データを2次元配列に挿入する
HaruyaData = []
for row in range(4, 24):
    song_names = mojimoji.zen_to_han(ws[f"D{row}"].value, kana=False).split('/')
    artist_name = mojimoji.zen_to_han(ws[f"C{row}"].value, kana=False)
    point = round(6.0 - ((row - 4) * 0.3), 2)  # 点数を計算する
    for song_name in song_names:
        if "/" in song_name:
            song_name_a, song_name_b = song_name.split("/")
            HaruyaData.append([song_name_a.strip(), artist_name, point])
            HaruyaData.append([song_name_b.strip(), artist_name, point])
        else:
            HaruyaData.append([song_name.strip(), artist_name, point])

# 2次元配列として表示
print("HaruyaData 読み込み完了")

HaruyaData 読み込み完了


**二次元配列に登録されたデータをtest.dbに入れる**

In [ ]:
# 二次元配列にある情報をデータベースに登録
# オリコン週間ランキング
for entry in OriconWeekData:
    title, artist, score = entry
    # 既存のデータがあればScoreを足して更新、なければ新規追加
    cursor.execute('''
        INSERT INTO music_master (Title, Artist, Score, Last_Rank, Last_Number, On_Chart)
        VALUES (?, ?, ?, 0, 0, 0)
        ON CONFLICT(Title, Artist) DO UPDATE SET Score = Score + ?
    ''', (title,artist, score, score))

# コミットして変更を保存
conn.commit()

In [ ]:
# オリコンデジタルランキング
for entry in OriconDigitalData:
    title, artist, score = entry
    # 既存のデータがあればScoreを足して更新、なければ新規追加
    cursor.execute('''
        INSERT INTO music_master (Title, Artist, Score, Last_Rank, Last_Number, On_Chart)
        VALUES (?, ?, ?, 0, 0, 0)
        ON CONFLICT(Title, Artist) DO UPDATE SET Score = Score + ?
    ''', (title, artist, score, score))

# コミットして変更を保存
conn.commit()

In [ ]:
# ビルボードJAPAN
for entry in BillboardData:
    title, artist, score = entry
    # 既存のデータがあればScoreを足して更新、なければ新規追加
    cursor.execute('''
        INSERT INTO music_master (Title, Artist, Score,Last_Rank, Last_Number, On_Chart)
        VALUES (?, ?, ?, 0, 0, 0)
        ON CONFLICT(Title, Artist) DO UPDATE SET Score = Score + ?
    ''', (title, artist, score, score))

# コミットして変更を保存
conn.commit()

In [ ]:
# 明屋書店
for entry in HaruyaData:
    title, artist, score = entry
    # 既存のデータがあればScoreを足して更新、なければ新規追加
    cursor.execute('''
        INSERT INTO music_master (Title, Artist, Score, Last_Rank, Last_Number, On_Chart)
        VALUES (?, ?, ?, 0, 0, 0)
        ON CONFLICT(Title, Artist) DO UPDATE SET Score = Score + ?
    ''', (title, artist, score, score))

# コミットして変更を保存
conn.commit()

**データベースから上位20個を出力する**

整形後もここの再生ボタンで出力してください

In [ ]:
# Scoreの高い順に上位25件を取得
top_20_query = '''
    SELECT * FROM music_master
    WHERE Score IS NOT NULL AND Score != ''
    ORDER BY Score DESC
    LIMIT 20
'''
top_20_results = cursor.execute(top_20_query).fetchall()
# コミットして変更を保存
conn.commit()

# 整形後のデータ出力用
df = pd.read_sql('''
SELECT * FROM music_master
WHERE Score IS NOT NULL AND Score != ''
ORDER BY Score DESC
LIMIT 60''', conn)

df.head(20)


,Title,Artist,Score,Last_Rank,Last_Number,On_Chart
0,LOVE TRIGGER,Snow Man,17.7,0,0,0
1,アンビバレント,Uru,13.8,17,2173,3
2,Bling-Bang-Bang-Born,Creepy Nuts,12.0,2,2174,4
3,FREEDOM,西川貴教 with t.komuro,11.7,1,2174,3
4,何色でもない花,宇多田ヒカル,10.8,0,0,0
5,唱,Ado,7.8,5,2174,21
6,幾億光年,Omoinotake,7.5,11,2174,2
7,After you!,キョコロヒー,7.5,0,0,0
8,アイドル,YOASOBI,6.9,8,2174,43
9,晩餐歌,tuki.,6.9,6,2174,6


**==========================ここまで=================================**

**ここから下はデータ整形用**

%はFM愛媛で言うレコード室の検索の＊と同じです

整形後のデータを確認したい場合は、この上の「データベースから上位20個を出力する」の再生ボタンを押してください

（反映されています）

**指定したタイトルの行を削除する**

同じ名前の楽曲名がある場合、両方が消えます。

In [ ]:
# 削除したいTitleを入力してください（%は消さないでください（前方一致検索））
cursor.execute('''DELETE FROM music_master
WHERE Title LIKE
 '大江裕';
 ''')


**指定したアーティストの行を削除する**

同じ名前のアーティストがあると、すべて消えます

**レコード室の検索方法のように「A」だけを入力して実行するとAから始まるアーティストすべてが消えます。**

なるべく完全一致での検索をお願いします


In [ ]:
cursor.execute('''DELETE FROM music_master
WHERE Artist LIKE
 'ケセラセラ%';
 ''')

**指定したタイトルのScoreを任意の点数追加する**

In [ ]:
# 削除したいTitleとScoreの値を入力してください（%は消さないでください（前方一致検索））
cursor.execute('''UPDATE music_master
SET Score = Score +5.7
WHERE Title LIKE
'光の跡%';
 ''')

**指定したタイトルのScoreを+3点する**

本来は「アーティスト」に3点ですが、アーティストに点数を追加するとそのアーティストすべての楽曲に３点が追加されてしまい下剋上が起きてしまうので、タイトルを指定してください

In [ ]:
# 3点追加したいTitleを入力してください（%は消さないでください（前方一致検索））
cursor.execute('''UPDATE music_master
SET Score = Score + 3
WHERE Title LIKE
'（指定したタイトル）%';
 ''')

**データベースに曲名・アーティスト名・Scoreを書き込む**

既にデータベースに曲がある場合は、上の任意の点数入力を使用してください（エラーが出ます）

In [ ]:
# 一番下の行に必要事項を入力してください
cursor.execute('''
        INSERT INTO music_master (Title, Artist, Score, Last_Rank, Last_Number, On_Chart)
        VALUES (?, ?, ?, 0, 0, 0);
    ''', ('ケセラセラ', 'Mrs.GREEN APPLE', 8.7))

IntegrityError: UNIQUE constraint failed: music_master.Title, music_master.Artist

**データベース整形コマンド集はここまで！**

**マジカルサイト送信用Excel自動生成**

In [ ]:
# def MajicalExcel():
from openpyxl.styles import Font, PatternFill
from openpyxl import load_workbook
from openpyxl.styles.alignment import Alignment

# Excelファイルを読み込み
workbook = load_workbook('ベストヒットランキング フォーマット.xlsx')
sheet = workbook.active

# 最終回の確認
# クエリの実行
query = "SELECT MAX(Last_Number) FROM music_master;"
cursor.execute(query)
# 結果の取得
max_last_number = cursor.fetchone()[0]
# 今回のベストヒットランキング回数の値を設定
print("ベストヒットランキングのNoを入力してください")
print("現在、DBに登録されている最新Noは"+str(max_last_number)+"です")
this_rank_number = int(input())
rank = 1
# 見出しのNo.と日付
sheet.cell(row=3, column=2).value = "Ｎｏ."+str(+this_rank_number) #No.の書き込み
sheet.merge_cells(start_row = 3 ,start_column=2 ,end_row =3 , end_column = 4) #セル結合

if sel == 1:
  sheet.cell(row=3, column=6).value = str(OriconTodays().year)+"年"+str(OriconTodays().month)+"月"+str(OriconTodays().day)+"日"#日付の書き込み

elif sel == 2:
  sheet.cell(row=3, column=6).value = str(OriconLastWeek().year)+"年"+str(OriconLastWeek().month)+"月"+str(OriconLastWeek().day)+"日"#日付の書き込み

else:
  sheet.cell(row=3, column=6).value = str(OriconSelectWeek().year)+"年"+str(OriconSelectWeek().month)+"月"+str(OriconSelectWeek().day)+"日"#日付の書き込み

# ランキングセルにデータを書き込み
for idx in range(1, len(top_20_results)*2, 2):
    result = top_20_results[idx // 2]  # idxを2で割った商を使って結果を取得
    title, artist, score, last_rank, last_number, on_chart= result

    # ランキング初登場曲
    # Last_Rankが空文字もしくはnullの場合、文字色を赤に変更
    if last_rank is None or last_rank == "" or last_rank == 0:
        sheet.cell(row=idx+5, column=2).value = rank
        sheet.cell(row=idx+5, column=2).font = Font(name ="BIZ UDPゴシック",size = 16,bold=True,color="FF0000") #ThisWeek
        sheet.alignment = Alignment(horizontal = 'center',vertical='center')
        sheet.merge_cells(start_row = idx +5 ,start_column=2 ,end_row = idx +6 , end_column = 2)
        sheet.cell(row=idx+5, column=3).value = "初"
        sheet.cell(row=idx+5, column=3).font = Font(name ="BIZ UDPゴシック",size = 16,bold=True,color="FF0000") #LastWeek
        sheet.merge_cells(start_row = idx +5 ,start_column=3 ,end_row = idx +6 , end_column = 3)
        sheet.cell(row=idx+5, column=4).value = "1"
        sheet.cell(row=idx+5, column=4).font = Font(name ="BIZ UDPゴシック",size = 16,bold=True,color="FF0000") #Onchart
        sheet.merge_cells(start_row = idx +5 ,start_column=4 ,end_row = idx +6 , end_column = 4)
        sheet.cell(row=idx+5, column=5).value = title
        sheet.cell(row=idx+5, column=6).value = artist

        rank += 1

    # ランキング再登場曲
    # Last_NumberがThis_Numberより2以上差がある場合、セルに「再」を入力
    elif (this_rank_number - last_number) >= 2 :
        sheet.cell(row=idx+5, column=2).value = rank#ThisWeek
        sheet.cell(row=idx+5, column=2).font = Font(name ="BIZ UDPゴシック",size = 16,bold=True,color="000000")
        sheet.merge_cells(start_row = idx +5 ,start_column=2 ,end_row = idx +6 , end_column = 2)
        sheet.cell(row=idx+5, column=3).value = "再"#LastWeek
        sheet.cell(row=idx+5, column=3).font = Font(name ="BIZ UDPゴシック",size = 16,bold=True,color="000000")
        sheet.merge_cells(start_row = idx +5 ,start_column=3 ,end_row = idx +6 , end_column = 3)
        sheet.cell(row=idx+5, column=4).value = on_chart+1
        sheet.cell(row=idx+5, column=4).font = Font(name ="BIZ UDPゴシック",size = 16,bold=True,color="000000")
        sheet.merge_cells(start_row = idx +5 ,start_column=4 ,end_row = idx +6 , end_column = 4)
        sheet.cell(row=idx+5, column=5).value = title
        sheet.cell(row=idx+5, column=6).value = artist

        rank+=1

    else:
    #  先週も登場した曲
    # セルにデータを書き込み
        sheet.cell(row=idx+5, column=5).value = title
        sheet.cell(row=idx+5, column=6).value = artist
        sheet.cell(row=idx+5, column=2).value = rank
        sheet.merge_cells(start_row = idx +5 ,start_column=2 ,end_row = idx +6 , end_column = 2)
        sheet.cell(row=idx+5, column=2).font = Font(name ="BIZ UDPゴシック",size = 16,bold=True,color="000000")
        sheet.cell(row=idx+5, column=3).value = last_rank
        sheet.cell(row=idx+5, column=3).font = Font(name ="BIZ UDPゴシック",size = 16,bold=True,color="000000")
        sheet.merge_cells(start_row = idx +5 ,start_column=3 ,end_row = idx +6 , end_column = 3)
        # sheet.cell(row=idx, column=7).value = last_number
        sheet.cell(row=idx+5, column=4).value = on_chart+1
        sheet.cell(row=idx+5, column=4).font = Font(name ="BIZ UDPゴシック",size = 16,bold=True,color="000000")
        sheet.merge_cells(start_row = idx +5 ,start_column=4 ,end_row = idx +6 , end_column = 4)
        rank+=1

# Excelファイルを保存

if sel == 1:
  workbook.save(str(OriconTodays())+'ベストヒットランキング.xlsx')

elif sel == 2:
  workbook.save(str(OriconLastWeek())+'ベストヒットランキング.xlsx')

else:
  workbook.save(str(OriconSelectWeek())+'ベストヒットランキング.xlsx')

print("正常に処理されました。下の2つのコードを実行すると、Excelデータがダウンロードできます")



# データベース接続を閉じる
# conn.close()

ベストヒットランキングのNoを入力してください
現在、DBに登録されている最新Noは2158です
2159
正常に処理されました。下の2つのコードを実行すると、Excelデータがダウンロードできます


**終了後はここから下を実行**

**楽曲データとExcelファイルダウンロード**

In [ ]:
# Excelファイルの読み込み
if sel==1:
  excel_file = str(OriconTodays())+'ベストヒットランキング.xlsx'
  workbook = openpyxl.load_workbook(excel_file)
  sheet = workbook.active


# データの処理と挿入
  for row in range(6, 45, 2):
    title = mojimoji.zen_to_han(sheet['E' + str(row)].value, kana=False)
    artist = mojimoji.zen_to_han(sheet['F' + str(row)].value, kana=False)
    rank = sheet['B' + str(row)].value
    on_chart = sheet['D'+str(row)].value
    this_number = mojimoji.zen_to_han(sheet['B3'].value,kana=False)
    this_number = this_number.replace('No.','')

    if "再" in str(sheet['C' + str(row)].value) or "圏外" in str(sheet['C' + str(row)].value) :
        last_number = int(this_number) - 1
    elif "初" in str(sheet['C' + str(row)].value):
        on_chart = 1
        last_number = this_number
    else:
        last_number = this_number

    # データベースに挿入
    cursor.execute('''INSERT INTO music_master
     (Title, Artist, Score, Last_Rank, Last_Number, On_chart)
     VALUES (?, ?, 0.0, ?, ?, ?)
     ON CONFLICT(Title, Artist) DO UPDATE SET Last_Number = ?,Last_Rank = ?,On_Chart = ?''',
            (title, artist, rank, last_number, on_chart,last_number,rank,on_chart))

  cursor.execute('UPDATE music_master SET Score = 0;') #Scoreの値を全消し

  cursor.execute('''DELETE FROM music_master WHERE Last_Number = '' OR Last_Number = 0;''') #Last_Numberが空文字もしくは0の場合はその曲を消す（ランクインしなかった曲）

 # コミットとクローズ
  conn.commit()

elif sel == 2:
  cursor.execute('UPDATE music_master SET Score = 0;') #Scoreの値を全消し
  cursor.execute('''DELETE FROM music_master WHERE Last_Number = '' OR Last_Number = 0;''') #Last_Numberが空文字もしくは0の場合はその曲を消す（ランクインしなかった曲）
  print("この処理は必要ありません。下へ進んでください")


else :
  cursor.execute('UPDATE music_master SET Score = 0;') #Scoreの値を全消し
  cursor.execute('''DELETE FROM music_master WHERE Last_Number = '' OR Last_Number = 0;''') #Last_Numberが空文字もしくは0の場合はその曲を消す（ランクインしなかった曲）
  print("この処理は必要ありません。下へ進んでください")




この処理は必要ありません。下へ進んでください


In [ ]:
import csv

with open('楽曲データ.csv', 'w', newline='') as f:
    # CSVライターオブジェクトを作成
    csv_writer = csv.writer(f)

    # データを取得
    cursor.execute("SELECT * FROM music_master")
    rows = cursor.fetchall()

    # データをCSVに書き込む
    for row in rows:
        csv_writer.writerow(row)

from google.colab import files
files.download('楽曲データ.csv')
if sel == 1:
 files.download(str(OriconTodays())+'ベストヒットランキング.xlsx')

elif sel == 2:
  files.download(str(OriconLastWeek())+'ベストヒットランキング.xlsx')

else:
  files.download(str(OriconSelectWeek())+'ベストヒットランキング.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**=====================================ここまで（ここから下は管理者用）=====================================**

**質が良すぎるデータづくり用**

2159回から

In [ ]:
import csv
import sqlite3

dbname = ('test.db')#データベース名.db拡張子で設定
conn = sqlite3.connect(dbname, isolation_level=None)#データベースを作成、自動コミット機能ON
cursor = conn.cursor() #カーソルオブジェクトを作成
sql = """CREATE TABLE music_master (
    Title TEXT,
    Artist TEXT,
    Score DOUBLE,
    Last_Rank INT,
    Last_Number INT,
    On_Chart INT,
    PRIMARY KEY (Title, Artist)
);"""
#↑DBのフォーマット設定（別のところに書いておきます）
cursor.execute(sql)#executeコマンドでSQL文を実行
conn.commit()#データベースにコミット(Excelでいう上書き保存。自動コミット設定なので不要だが一応・・)
with open('楽曲データ.csv', 'r') as f:
    # CSVリーダーオブジェクトを作成
    csv_reader = csv.reader(f)
    for row in csv_reader:
        # テーブルにデータを挿入
        cursor.execute("INSERT INTO music_master VALUES (?,?,?,?,?,?)", (row[0], row[1],row[2],row[3],row[4],row[5]))

In [ ]:
# Excelファイルの読み込み
excel_file = '3ベストヒットえひめランキングHP用.xlsx'
workbook = openpyxl.load_workbook(excel_file)
sheet = workbook.active


# データの処理と挿入
for row in range(6, 45, 2):
    title = mojimoji.zen_to_han(sheet['E' + str(row)].value, kana=False)
    artist = mojimoji.zen_to_han(sheet['F' + str(row)].value, kana=False)
    rank = sheet['B' + str(row)].value
    on_chart = sheet['D'+str(row)].value
    this_number = mojimoji.zen_to_han(sheet['B3'].value,kana=False)
    this_number = this_number.replace('No.','')

    if "再" in str(sheet['C' + str(row)].value) or "圏外" in str(sheet['C' + str(row)].value) :
        last_number = int(this_number) - 1
    elif "初" in str(sheet['C' + str(row)].value):
        on_chart = 1
        last_number = this_number
    else:
        last_number = this_number

    # データベースに挿入
    cursor.execute('''INSERT INTO music_master
     (Title, Artist, Score, Last_Rank, Last_Number, On_chart)
     VALUES (?, ?, 0.0, ?, ?, ?)
     ON CONFLICT(Title, Artist) DO UPDATE SET Last_Number = ?,Last_Rank = ?,On_Chart = ?''',
            (title, artist, rank, last_number, on_chart,last_number,rank,on_chart))

# コミットとクローズ
conn.commit()

FileNotFoundError: [Errno 2] No such file or directory: '3ベストヒットえひめランキングHP用.xlsx'

In [ ]:
df = pd.read_sql('''SELECT * FROM music_master ORDER BY Last_Number DESC;''',conn)
df.head(60)

,Title,Artist,Score,Last_Rank,Last_Number,On_Chart
0,アイドル,YOASOBI,0.0,6,2158,25
1,青のすみか,キタニタツヤ,0.0,7,2158,13
2,SPECIALZ,King Gnu,0.0,1,2158,6
3,Dangerholic,SnowMan,0.0,5,2158,5
4,唱,Ado,0.0,2,2158,5
5,Make Up Day,なにわ男子,0.0,11,2158,3
6,本音と建前,Sexy Zone,0.0,8,2158,3
7,アイドルなんかじゃなかったら,AKB48,0.0,18,2158,2
8,勇者,YOASOBI,0.0,3,2158,2
9,Anytime Anywhere,milet,0.0,13,2158,2


In [ ]:
cursor.execute('UPDATE music_master SET Score = 0;') #Scoreの値を全消し

cursor.execute('DELETE FROM music_master WHERE Last_Number = '' OR Last_Number = 0;') #Last_Numbweが空文字ものを全消し

In [ ]:
import csv

with open('楽曲データ.csv', 'w', newline='') as f:
    # CSVライターオブジェクトを作成
    csv_writer = csv.writer(f)

    # データを取得
    cursor.execute("SELECT * FROM music_master")
    rows = cursor.fetchall()

    # データをCSVに書き込む
    for row in rows:
        csv_writer.writerow(row)

from google.colab import files
files.download('楽曲データ.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**github（準備中）**

In [ ]:
%cd /content/JOEU-FM/
!git clean -f
!git clone https://github.com/super-onigiri-man/JOEU-FM
!git config --global user.email "syunsuke030828@gmail.com"
!git config --global user.name "super-onigiri-man"
!git add .
!git commit -m "楽曲データを更新"
!git remote set-url origin https://github.com/super-onigiri-man/JOEU-FM.git
!git push origin main

/content/JOEU-FM
Cloning into 'JOEU-FM'...
remote: Enumerating objects: 234, done.
remote: Counting objects: 100% (234/234), done.
remote: Compressing objects: 100% (130/130), done.
remote: Total 234 (delta 134), reused 154 (delta 101), pack-reused 0
Receiving objects: 100% (234/234), 247.04 KiB | 7.06 MiB/s, done.
Resolving deltas: 100% (134/134), done.
hint: You've added another git repository inside your current repository.
hint: Clones of the outer repository will not contain the contents of
hint: the embedded repository and will not know how to obtain it.
hint: If you meant to add a submodule, use:
hint: 
hint: 	git submodule add <url> JOEU-FM
hint: 
hint: If you added this path by mistake, you can remove it from the
hint: index with:
hint: 
hint: 	git rm --cached JOEU-FM
hint: 
hint: See "git help submodule" for more information.
[main fe59ee0] 楽曲データを更新
 1 file changed, 1 insertion(+)
 create mode 160000 JOEU-FM
fatal: could not read Username for 'https://github.com': No such dev

**DB壊れた時用**

In [ ]:
dbname = ('test.db')#データベース名.db拡張子で設定
conn = sqlite3.connect(dbname, isolation_level=None)#データベースを作成、自動コミット機能ON
cursor = conn.cursor() #カーソルオブジェクトを作成
sql = """CREATE TABLE music_master (
    Title TEXT,
    Artist TEXT,
    Score DOUBLE,
    Last_Rank INT,
    Last_Number INT,
    On_Chart INT,
    PRIMARY KEY (Title, Artist)
);"""
#↑DBのフォーマット設定（別のところに書いておきます）
cursor.execute(sql)#executeコマンドでSQL文を実行
conn.commit()#データベースにコミット(Excelでいう上書き保存。自動コミット設定なので不要だが一応・・)